### 주식 정보 가져오기
- url = https://finance.naver.com/item/sise_day.nhn?code=003490&page=1
- url 로부터 html 가져오기 requests
- 가져온 html로부터 분석하기 BeautifulSoup
- 해당객체의 정보 가져오기

In [7]:
import requests
from bs4 import BeautifulSoup

In [6]:
url = 'https://finance.naver.com/item/sise_day.nhn?code=003490&page=1'

In [8]:
req = requests.get(url)
req

<Response [200]>

In [9]:
req.text[:100]

'\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n'

In [10]:
html = req.text
html[:200]

'\n\n\n\n\n\n\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n<link rel="stylesheet" type="text/css" href="/css/newstock.css?20200806200958"'

In [12]:
soup = BeautifulSoup(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [18]:
data = soup.select('table.type2 > tr')
len(data)

16

In [35]:
# ohlc(open-high-low-close)
for d in data:
    tgt = d.select('td')
    n=len(d.select('td.num'))
    if(n>0):
        sdate = tgt[0].get_text().replace(".","")
        endp = tgt[1].get_text().replace(",","")
        startp = tgt[3].get_text().replace(",","")
        maxp = tgt[4].get_text().replace(",","")
        minp = tgt[5].get_text().replace(",","")
        vol = tgt[6].get_text().replace(",","")
        t = [int(sdate), int(startp), int(maxp), int(minp), int(endp), int(vol)]
        print(t)

[20200812, 19250, 20100, 19050, 19550, 8846552]
[20200811, 19350, 19550, 18850, 18900, 3632851]
[20200810, 19000, 19100, 18350, 18750, 3821098]
[20200807, 20500, 20950, 18850, 19100, 15844472]
[20200806, 17400, 18200, 17350, 18100, 5637601]
[20200805, 17400, 17500, 17100, 17250, 1410989]
[20200804, 17100, 17400, 17050, 17350, 1176523]
[20200803, 17350, 17350, 17000, 17150, 1738510]
[20200731, 17400, 17500, 17100, 17400, 1733960]
[20200730, 17500, 17650, 17300, 17400, 1859030]


In [37]:
import cx_Oracle

In [39]:
uid = 'PROJECT1'
upw = 'Test1234'
url = 'localhost:1521/orcl'
conn = cx_Oracle.connect(uid,upw,url)
print(conn)

<cx_Oracle.Connection to PROJECT1@localhost:1521/orcl>


In [97]:
#data = [20200812, 19250, 20100, 19050, 19550, 8846552]
def insertStock(data=[]):
    sql = "insert into STOCKNAV (SIDX,SDATE,OPENP,HIGHP,LOWP,CLOSEP,VOL,SSTAT) values(STOCK_SEQ.NEXTVAL,{0},{1},{2},{3},{4},{5},0)"
    sql1= sql.format(*data)
    
    try:
        cursor.execute(sql1)
        return(1)
    except cx_Oracle.DatabaseError as exc:
        error, = exc.args
        print("Oracle-Error-Code:", error.code)
        print("Oracle-Error-Message:", error.message)
        return(0)
    
insertStock(data)



Oracle-Error-Code: 1
Oracle-Error-Message: ORA-00001: 무결성 제약 조건(PROJECT1.STOCKNAV_UK1)에 위배됩니다


0

In [80]:
data = [20200810, 19250, 20100, 19050, 19550, 8846552]
def addData(data):
    cursor = conn.cursor()
    sql = "select * from STOCKNAV where SDATE="+str(data[0])
    cursor.execute(sql)
    res = cursor.fetchall()
    #print(res[0][1])
    if (len(res)>0):
        print("데이터 중복")        
    else:
        print("데이터 입력가능")
        insertStock(data)
        conn.commit()
        
addData(data)

데이터 입력가능
